# Gaze Heatmap Batch Processor - Google Colab
Process eye-tracking gaze data and generate heatmap crops for multiple users in NEAR Experiment Design dataset

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import required libraries
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path

In [7]:
# Set file paths
BASE_SOURCE_PATH = "/content/drive/MyDrive/NEAR_Experiment_Design/PilotData_V1_10232025"
BASE_OUTPUT_PATH = "/content/drive/MyDrive/NEAR_Experiment_Design_Output"

# Create output directory if it doesn't exist
os.makedirs(BASE_OUTPUT_PATH, exist_ok=True)

# List all user folders in source path (skip hidden/system folders)
user_folders = sorted([f for f in os.listdir(BASE_SOURCE_PATH) 
                       if os.path.isdir(os.path.join(BASE_SOURCE_PATH, f)) 
                       and not f.startswith('.')
                       and f not in ['1_Data_Analysis']])  # Skip non-user folders
print(f"Found {len(user_folders)} user folders:")
for folder in user_folders[:10]:  # Show first 10
    print(f"  - {folder}")
if len(user_folders) > 10:
    print(f"  ... and {len(user_folders) - 10} more")

Found 26 user folders:
  - AT_1
  - AT_2
  - AT_3_1
  - AT_3_2
  - Ayu_1
  - Ayu_2
  - Ayu_3
  - JC_1
  - JC_2
  - JC_3_1
  ... and 16 more


In [4]:
# Helper functions for heatmap processing

def open_cap(path):
    """Open video and extract metadata"""
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise IOError(f"cannot open video {path}")
    fps = cap.get(cv2.CAP_PROP_FPS)
    count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = count / fps
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cap, fps, count, duration, w, h

def gaussian_heatmap(size, points, sigma=50):
    """Create gaussian heatmap from gaze points"""
    heat = np.zeros(size, dtype=np.float32)
    for (x, y) in points:
        if 0 <= x < size[1] and 0 <= y < size[0]:
            heat[y, x] += 1
    heat = cv2.GaussianBlur(heat, (0, 0), sigma)
    return heat

def overlay_heatmap_masked(src_frame, hmap_normalized, alpha=0.5):
    """Overlay heatmap on source with alpha masking"""
    hm_8bit = (hmap_normalized * 255).astype(np.uint8)
    hm_colored = cv2.applyColorMap(hm_8bit, cv2.COLORMAP_JET)
    alpha_mask = np.stack([hmap_normalized * alpha, hmap_normalized * alpha, hmap_normalized * alpha], axis=2)
    overlaid = (src_frame * (1 - alpha_mask) + hm_colored * alpha_mask).astype(np.uint8)
    return overlaid

def bbox_from_heatmap_only(hmap, pad=0, thresh=0.5):
    """Extract bounding box from heatmap"""
    mask = hmap > thresh
    ys, xs = np.where(mask)
    if len(xs) == 0:
        return None
    x0, x1 = xs.min(), xs.max()
    y0, y1 = ys.min(), ys.max()
    x0, y0 = max(0, x0 - pad), max(0, y0 - pad)
    x1, y1 = min(hmap.shape[1], x1 + pad), min(hmap.shape[0], y1 + pad)
    return x0, y0, x1, y1

In [5]:
# Main processing function

def process_user_folder(user_name, source_base_path, output_base_path, 
                        interval_sec=3.0, pad=20, alpha=0.5, sample_windows=None):
    """Process a single user's eye-tracking data"""
    
    user_source_path = os.path.join(source_base_path, user_name)
    user_output_path = os.path.join(output_base_path, user_name)
    
    # Construct paths
    source_video = os.path.join(user_source_path, "world.mp4")
    gaze_csv = os.path.join(user_source_path, "exports", "000", "gaze_positions.csv")
    
    # Check if required files exist
    if not os.path.exists(source_video):
        print(f"⚠️  {user_name}: world.mp4 not found")
        return False
    if not os.path.exists(gaze_csv):
        print(f"⚠️  {user_name}: gaze_positions.csv not found")
        return False
    
    # Create output directories
    frames_dir = os.path.join(user_output_path, "frames")
    os.makedirs(frames_dir, exist_ok=True)
    
    print(f"📊 Processing {user_name}...")
    
    # Open source video
    try:
        src_cap, src_fps, src_count, src_dur, src_w, src_h = open_cap(source_video)
    except Exception as e:
        print(f"❌ {user_name}: Failed to open video - {e}")
        return False
    
    # Load gaze data
    try:
        gaze_df = pd.read_csv(gaze_csv)
        if 'gaze_timestamp' in gaze_df.columns:
            tscol = 'gaze_timestamp'
        elif 'timestamp' in gaze_df.columns:
            tscol = 'timestamp'
        else:
            raise ValueError("No timestamp column found")
        
        gaze_df = gaze_df.rename(columns={tscol: 'timestamp'})
        gaze_df = gaze_df.dropna(subset=['norm_pos_x', 'norm_pos_y'])
        gaze_df['timestamp'] = gaze_df['timestamp'] - gaze_df['timestamp'].min()
    except Exception as e:
        print(f"❌ {user_name}: Failed to load gaze data - {e}")
        src_cap.release()
        return False
    
    # Process intervals
    num = int(src_dur // interval_sec)
    if sample_windows is not None:
        num = min(num, sample_windows)
    
    interval_frames = int(interval_sec * src_fps)
    processed = 0
    
    for k in range(num):
        try:
            start_t = k * interval_sec
            end_t = start_t + interval_sec
            mid_t = start_t + interval_sec / 2
            mid_frame = int(mid_t * src_fps)
            
            # Read source frame
            src_cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame)
            ret, src_frame = src_cap.read()
            if not ret:
                continue
            
            # Save full source frame
            cv2.imwrite(os.path.join(frames_dir, f"src_{k}.png"), src_frame)
            
            # Compute heatmap from gaze data
            mask = (gaze_df['timestamp'] >= start_t) & (gaze_df['timestamp'] < end_t)
            df_win = gaze_df.loc[mask]
            
            if not df_win.empty:
                xs = (df_win['norm_pos_x'].values * src_w).astype(int)
                ys = (df_win['norm_pos_y'].values * src_h).astype(int)
                pts = np.vstack((xs, ys)).T
                hmap = gaussian_heatmap((src_h, src_w), pts)
                
                if hmap.max() > 0:
                    hmap = hmap / hmap.max()
                    
                    # Get bounding box
                    bbox = bbox_from_heatmap_only(hmap, pad=pad)
                    if bbox is not None:
                        x0, y0, x1, y1 = bbox
                        
                        # Save overlaid heatmap
                        overlaid_frame = overlay_heatmap_masked(src_frame, hmap, alpha=alpha)
                        cv2.imwrite(os.path.join(frames_dir, f"heat_{k}.png"), overlaid_frame)
                        
                        # Save cropped versions
                        crop_overlaid = overlaid_frame[y0:y1, x0:x1]
                        cv2.imwrite(os.path.join(frames_dir, f"heat_{k}_crop.png"), crop_overlaid)
                        
                        crop_src = src_frame[y0:y1, x0:x1]
                        cv2.imwrite(os.path.join(frames_dir, f"src_{k}_crop.png"), crop_src)
                        
                        processed += 1
        except Exception as e:
            print(f"  ⚠️  Interval {k}: {e}")
            continue
    
    src_cap.release()
    print(f"✅ {user_name}: Processed {processed}/{num} intervals")
    return True

In [8]:
# Configuration
INTERVAL_SEC = 3.0
PAD = 20
ALPHA = 0.5  # Heatmap transparency (0.1-0.9)
SAMPLE_WINDOWS = None  # None for full video, or set to a number to limit intervals

# Test with one user first
test_user = user_folders[0] if user_folders else None
if test_user:
    print(f"Testing with user: {test_user}\n")
    process_user_folder(test_user, BASE_SOURCE_PATH, BASE_OUTPUT_PATH, 
                       interval_sec=INTERVAL_SEC, pad=PAD, alpha=ALPHA, sample_windows=SAMPLE_WINDOWS)

Testing with user: AT_1

📊 Processing AT_1...
✅ AT_1: Processed 23/23 intervals


In [10]:
# Process all users (uncomment and run when ready)
print("\n" + "="*60)
print("BATCH PROCESSING ALL USERS")
print("="*60 + "\n")

successful = 0
failed = 0

for user in user_folders:
    try:
        result = process_user_folder(user, BASE_SOURCE_PATH, BASE_OUTPUT_PATH, 
                                    interval_sec=INTERVAL_SEC, pad=PAD, alpha=ALPHA, sample_windows=SAMPLE_WINDOWS)
        if result:
            successful += 1
        else:
            failed += 1
    except Exception as e:
        print(f"❌ {user}: Unexpected error - {e}")
        failed += 1

print("\n" + "="*60)
print(f"BATCH PROCESSING COMPLETE")
print(f"✅ Successful: {successful}")
print(f"❌ Failed: {failed}")
print(f"📁 Output location: {BASE_OUTPUT_PATH}")
print("="*60)


BATCH PROCESSING ALL USERS

📊 Processing AT_1...
✅ AT_1: Processed 23/23 intervals
📊 Processing AT_2...
✅ AT_2: Processed 9/9 intervals
📊 Processing AT_3_1...
✅ AT_3_1: Processed 6/6 intervals
📊 Processing AT_3_2...
✅ AT_3_2: Processed 14/14 intervals
📊 Processing Ayu_1...
✅ Ayu_1: Processed 19/19 intervals
📊 Processing Ayu_2...
✅ Ayu_2: Processed 13/13 intervals
📊 Processing Ayu_3...
✅ Ayu_3: Processed 25/25 intervals
📊 Processing JC_1...
✅ JC_1: Processed 18/18 intervals
📊 Processing JC_2...
✅ JC_2: Processed 31/31 intervals
📊 Processing JC_3_1...
✅ JC_3_1: Processed 9/9 intervals
📊 Processing JC_3_2...
✅ JC_3_2: Processed 13/13 intervals
📊 Processing KC_1...
✅ KC_1: Processed 9/9 intervals
📊 Processing KC_2...
✅ KC_2: Processed 15/15 intervals
📊 Processing KC_3_1...
✅ KC_3_1: Processed 5/5 intervals
📊 Processing KC_3_2...
✅ KC_3_2: Processed 5/5 intervals
📊 Processing LKH_1...
✅ LKH_1: Processed 12/12 intervals
📊 Processing LKH_2...
✅ LKH_2: Processed 13/13 intervals
📊 Processing L

In [9]:
# Utility: Check generated files for a user
def check_output_files(user_name):
    """List generated output files for a user"""
    user_output_path = os.path.join(BASE_OUTPUT_PATH, user_name)
    frames_dir = os.path.join(user_output_path, "frames")
    
    if not os.path.exists(frames_dir):
        print(f"No output found for {user_name}")
        return
    
    files = sorted(os.listdir(frames_dir))
    src_full = [f for f in files if f.startswith("src_") and "_crop" not in f]
    src_crop = [f for f in files if "src_" in f and "_crop" in f]
    heat_full = [f for f in files if f.startswith("heat_") and "_crop" not in f]
    heat_crop = [f for f in files if "heat_" in f and "_crop" in f]
    
    print(f"\n📁 Output files for {user_name}:")
    print(f"  Source full frames: {len(src_full)}")
    print(f"  Source crops:       {len(src_crop)}")
    print(f"  Heat full frames:   {len(heat_full)}")
    print(f"  Heat crops:         {len(heat_crop)}")
    print(f"  Total files:        {len(files)}")

# Example: check output for first user
if user_folders:
    check_output_files(user_folders[0])


📁 Output files for AT_1:
  Source full frames: 23
  Source crops:       23
  Heat full frames:   23
  Heat crops:         23
  Total files:        92
